# Overview

Run this script to generate ERA5 and SARAH cutouts using atlite. The two weather data sources used by atlite are:

 - ERA5 (wind and solar)
   - ERA5 data can be downloaded through the Copernicus Climate Data Store cdsapi package, and requires 
registration and setup as per https://cds.climate.copernicus.eu/api-how-to
 - SARAH (solar)
   - SARAH data can be downloaded via from EUMETSAT CMSAF using the portal https://wui.cmsaf.eu/safira/action/viewProduktSearch
Surface Incoming Direct Radiation (SID), Surface Incoming Shortwave Radiation (SIS) and Direct Normal Irradiance (DNI) should be downloaded
and extracted into the sarah_dir path

In [1]:
import atlite

# Prepare ERA5 cutout

In [2]:
cutout_era5 = atlite.Cutout(
    path="../../cutouts/RSA-2010_22-era5.nc",
    #module=["era5"],
    #x=slice(16, 35),
    #y=slice(-35, -21),
    #time=slice("2020","2021"), # e.g. download 2 years of weather data
    chunks={"time": 100},
)

In [3]:
cutout_era5.data = cutout_era5.data.sel(time=slice("2012","2012"))

# Prepare SARAH cutout

In [7]:
cutout_sarah = atlite.Cutout(
    path="../../cutouts/RSA-2012-sarah.nc",
    #module=["sarah"],
    sarah_dir="../../cutouts/sarah_v3",
    #x=slice(16, 35),
    #y=slice(-35, -21),
    #time=slice("2012","2012"),
    chunks={"time": 1000}, # temp fix chunking by lat-1 to resolve error on building the cutout
)
#cutout_sarah.prepare(features="influx")

/home/pklein/anaconda3/envs/atlite/lib/python3.12/site-packages/atlite/cutout.py:190: UserWarning: Arguments sarah_dir are ignored, since cutout is already built.
  warn(


In [8]:
# Merge Sarah and ERA5 cutouts if prepared separately
cutout_sarah.data = cutout_sarah.data.interp(y=cutout_era5.grid.y.unique(), x=cutout_era5.grid.x.unique(), method="linear")
cutout_sarah = cutout_sarah.merge(cutout_era5, compat="override")
cutout_sarah.data.to_netcdf("RSA-2012-sarah.nc")


In [38]:
# Merge cutouts for multiple years

cutout_sarah = atlite.Cutout(
    path="../../cutouts/RSA-2012-sarah.nc",
    chunks={"time": 1000},
)

for y in [2018, 2019, 2020, 2021, 2022]:
    print(y)
    cutout_sarah = cutout_sarah.merge(
        atlite.Cutout(
            path=f"../../cutouts/RSA-{y}-sarah.nc",
            chunks={"time": 1000}
        )
    )
    
cutout_sarah.data = cutout_sarah.data.interp(y=cutout_era5.grid.y.unique(), x=cutout_era5.grid.x.unique(), method="linear")
cutout_sarah = cutout_sarah.merge(cutout_era5, compat="override")

cutout_sarah.data.to_netcdf("RSA-2017_22-sarah.nc")


2018
2019
2020
2021
2022
